In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Concatenate, Flatten, BatchNormalization, Dropout
from tensorflow.keras.activations import sigmoid, relu, tanh
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers, regularizers

import pickle
import matplotlib.pyplot as plt

In [ ]:
from module import ml_model as ml
from module import util
from module import data_transform as transform
from module import data_generator as gen

In [ ]:
# Load the configuration
config = util.load_config("../configs/config.yaml")

In [ ]:
# Load the training data
with open('../training_data/3_CHs_300_phases_500000_samples.pickle', 'rb') as read:
    data_loaded = pickle.load(read)

f = data_loaded["data"][0]
r = data_loaded["data"][1]
a = data_loaded["data"][2]

In [ ]:
data = list(zip(f, r, a))
# shuffle data in a same order
np.random.shuffle(data)
# unzip the data
f, r, a = zip(*data)

f = np.array(f)
r = np.array(r)
a = np.array(a)

In [ ]:
num_samples = 400000
split = int(num_samples * 0.8)

# Load the training and testing data
F_train = f[:split]
R_train = r[:split]
A_train = a[:split]

F_val = f[split:num_samples]
R_val = r[split:num_samples]
A_val = a[split:num_samples] 

In [ ]:
A_train = gen.generate_A_from_cascasded_AoA(A_train, 300)
A_val = gen.generate_A_from_cascasded_AoA(A_val, 300)

In [ ]:
print(F_train.shape, R_train.shape, A_train.shape)
print(F_val.shape, R_val.shape, A_val.shape)
config_training = data_loaded["config"]

In [ ]:
# data transformer
data_transformer = transform.DataNormalization()
data_transformer.fit(F_train, R_train)
print(vars(data_transformer))

In [ ]:
# Create a DNN model
DNN = ml.DNN(config)
DNN.data_transformer = data_transformer
DNN.construct_DNN_model()
# Train and save model periodically
DNN.fit(F_train=F_train, R_train=R_train, A_train=A_train, F_val=F_val, R_val=R_val, A_val=A_val, cascaded_data=False)

In [ ]:
plt.plot(DNN.loss_for_training)
plt.plot(DNN.loss_for_val)
plt.show()